In [ ]:
import json
from pathlib import Path
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score, f1_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.cluster import AgglomerativeClustering, KMeans
import optuna
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

BASE_DIR = Path().resolve()

<h2>Tidying Up</h2>

In [ ]:
df = pd.read_csv(BASE_DIR / "natality_7yr_test_data.csv")

df.dtypes

In [ ]:
df.nunique()

In [ ]:
binary_cols = [col for col in df.columns if df[col].nunique() == 2 and df[col].dtype == 'int64']

binary_cols

In [ ]:
df['imp_sex'].unique()

In [ ]:
df = df.drop('imp_sex', axis=1)

df['date'] = pd.to_datetime(df['dob_yy'].astype(str) + df['dob_wk'].astype(str) + '1', format='%G%V%u')

# Fourier terms for time of day as a type of "seasonality"
df['time_str'] = df['dob_tt'].astype(str).str.zfill(4)
df['hour'] = df['time_str'].str[:2].astype(int)
df['minute'] = df['time_str'].str[2:].astype(int)

df['minute_of_day'] = df['hour'] * 60 + df['minute']
df['time_sin'] = np.sin(2 * np.pi * df['minute_of_day'] / 1440)
df['time_cos'] = np.cos(2 * np.pi * df['minute_of_day'] / 1440)

# Monthly Fourier terms for monthly "seasonality"
# df['month_sin'] = np.sin(2 * np.pi * df['dob_mm'] / 12)
# df['month_cos'] = np.cos(2 * np.pi * df['dob_mm'] / 12)

df = df.drop(['dob_yy', 'dob_tt', 'dob_wk', 'time_str', 'minute_of_day', 'minute', 'hour'], axis=1)

df['sex'] = np.where(df['sex'] == 'M', 1, 0)

In [ ]:
df['no_mmorb'].value_counts()

In [ ]:
# clean outcome and drop where we don't have a label (we aren't at risk of data-shortage lol)

df = df.rename(columns={'no_mmorb': 'death_reported'})

df = df[df['death_reported'] != 9]
# flipping the binary so mother's death is the positive class
df['death_reported'] = 1 - df['death_reported']

In [ ]:
#[col for col in df.columns if col not in binary_cols]

List of variables that require some cleaning in their encodings - lots of sentinel values.

- fagecomb - 99 is a sentinel value; will need to drop or fill it
- priorlive - 99 is a sentinel value
- priordead - 99 sentinel value
- priorterm - 99
- *illb_r (Interval Since Last Live Birth Recode) - 000–003 are sentinel; 888 - Not applicable — first live birth; 999 - Unknown or not stated
- ilop_r (Interval Since Last Other Pregnancy Recode) - not sure if we want to use; same sentinel as illb_r
- *ilp_r (Interval Since Last Pregnancy Recode) - not sure we want to use; same sentinel as illb_r
- ilp_r11 (Interval Since Last Pregnancy Recode 11) - same as above
- precare - 99
- previs - 99
- cig_0 - cig_3 - 99
- bmi - 99.9
- pwgt_r - 999
- dwgt_r - 999
- wtgain - 99
- rf_cesarn - 99
- combgest - 99
- dbwt - 999


In [ ]:
cols_99 = [
    'fagecomb', 'priorlive', 'priordead', 'priorterm',
    'precare', 'previs', 'cig_1', 'cig_2', 'cig_3',
    'wtgain', 'rf_cesarn', 'combgest'
]
df[cols_99] = df[cols_99].replace(99, np.nan)

df[['pwgt_r', 'dwgt_r', 'dbwt']] = df[['pwgt_r', 'dwgt_r', 'dbwt']].replace(999, np.nan)

df['bmi'] = df['bmi'].replace(99.9, np.nan)
df.loc[df['bmi'] > 90, 'bmi'] = np.nan

df[['illb_r', 'ilp_r']] = df[['illb_r', 'ilp_r']].replace(
    [0, 1, 2, 3, 888, 999],
    np.nan
)

df[['ca_down', 'ca_disor']] = df[['ca_down', 'ca_disor']].replace('C', 2).astype(int)

# These seem redundant, so removed for now - can always comment out.
df = df.drop(columns=['ilop_r', 'ilp_r11'], axis=1)

In [ ]:
continuous_cols = [ # 'illb_r', 'ilp_r' <- dropped later on due to too many missing values
    'mager', 'fagecomb', 'priorlive', 'priordead', 'priorterm',
    'precare', 'previs', 'cig_0', 'cig_1', 'cig_2',
    'cig_3', 'bmi', 'pwgt_r', 'dwgt_r', 'wtgain', 'rf_cesarn', 'combgest',
    'dbwt', 'time_sin', 'time_cos' #, 'month_sin', 'month_cos'
]
cat_cols = [
    'bfacil', 'mracehisp', 'mar_p', 'meduc', 'fracehisp', 'feduc',
    'cig_rec', 'rf_pdiab', 'rf_gdiab', 'rf_phype', 'rf_ghype', 
    'rf_ehype', 'rf_ppterm', 'rf_inftr', 'rf_fedrg', 'rf_artec',
    'rf_cesar', 'ip_gon', 'ip_syph', 'ip_chlam', 'ip_hepb', 'ip_hepc',
    'ld_indl', 'ld_augm', 'ld_ster', 'ld_antb', 'ld_chor', 'ld_anes',
    'me_pres', 'me_rout', 'me_trial', 'mm_mtr', 'mm_plac', 'mm_rupt',
    'mm_uhyst', 'mm_aicu', 'attend', 'pay', 'dplural',
    'ab_aven1', 'ab_aven6', 'ab_nicu', 'ab_surf', 'ab_anti', 'ab_seiz',
    'ca_down', 'ca_disor', 'dob_mm' # using monthly seasonality as one-hot instead of Fourier. Will be more descriptive.
]

In [ ]:
enc = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

for col in cat_cols:
    col_data = df[col].astype("string")

    encoded = enc.fit_transform(col_data.to_frame())
    encoded_cols = enc.get_feature_names_out([col])
    encoded_df = pd.DataFrame(encoded, columns=encoded_cols, index=df.index)

    df = pd.concat([df, encoded_df], axis=1)
    df = df.drop(columns=[col])

Variables we're dropping to form the 'baseline' or intercept in Logistic Regression. We don't have to one-hot if we end up using a tree method for anything.

- bfacil_1 - Hospital
- mracehisp_1 - Non-Hispanic White (only)
- mar_p_1 - Yes
- meduc_1 - 8th grade or less
- fracehisp_1 - Non-Hispanic White (only)
- feduc_1 - 8th grade or less
- cig_rec_0 - No
- rf_pdiab_0 - No
- rf_gdiab_0 - No
- rf_phype_0 - No
- rf_ghype_0 - No
- rf_ehype_0 - No
- rf_ppterm_0 - No
- rf_inftr_0 - No
- rf_fedrg_0 - No
- rf_artec_0 - No
- rf_cesar_0 - No
- ip_gon_0 - No
- ip_syph_0 - No
- ip_chlam_0 - No
- ip_hepb_0 - No
- ip_hepc_0 - No
- ld_indl_0 - No
- ld_augm_0 - No
- ld_ster_0 - No
- ld_antb_0 - No
- ld_chor_0 - No
- ld_anes_0 - No
- me_pres_1 - Cephalic
- me_rout_1 - Spontaneous
- me_trial_0 - No
- mm_mtr_0 - No
- mm_rupt_0 - No
- mm_uhyst_0 - No
- mm_aicu_0 - No
- attend_1 - Doctor of Medicine (MD)
- pay_2 - Private Insurance
- dplural_1 - Single
- ab_aven1_0 - No
- ab_aven6_0 - No
- ab_nicu_0 - No
- ab_surf_0 - No
- ab_anti_0 - No
- ab_seiz_0 - No
- ca_down_0 - No
- ca_disor_0 - No
- dob_mm_1 - January

In [ ]:
df = df.drop(columns=['bfacil_1', 'mracehisp_1', 'mar_p_1', 'meduc_1', 'fracehisp_1', 'feduc_1', 'cig_rec_0', 'rf_pdiab_0', 
                 'rf_gdiab_0', 'rf_phype_0', 'rf_ghype_0', 'rf_ehype_0', 'rf_ppterm_0', 'rf_fedrg_0', 'rf_artec_0',
                 'rf_cesar_0', 'ip_gon_0', 'ip_syph_0', 'ip_chlam_0', 'ip_hepb_0', 'ip_hepc_0', 'ld_indl_0', 'ld_augm_0',
                 'ld_ster_0', 'ld_antb_0', 'ld_chor_0', 'ld_anes_0', 'me_pres_1', 'me_rout_1', 'me_trial_0', 'mm_mtr_0',
                 'mm_rupt_0', 'mm_uhyst_0', 'mm_aicu_0', 'pay_2', 'dplural_1', 'ab_aven1_0', 'ab_aven6_0',
                 'ab_nicu_0', 'ab_surf_0', 'ab_anti_0', 'ab_seiz_0', 'dob_mm_1'
])

In [ ]:
missing_df = (
    df.isna()
      .sum()
      .sort_values(ascending=False)
      .reset_index()
      .set_axis(['variable', 'missing'], axis=1)
)
missing_df['missing_pct'] = missing_df['missing'] / len(df)

missing_df

Drop features that have a count of missing values above a specific threshold. Impute based on the median of the data points' given year of measurement otherwise.

In [ ]:
threshold = 0.20

df = df.drop(columns=missing_df[missing_df['missing_pct'] > threshold]['variable'], axis=1)

for var in missing_df['variable']:
    if var not in df.columns:
        continue

    missing_pct = missing_df.loc[missing_df['variable'] == var, 'missing_pct'].values[0]

    if missing_pct > 0 and df[var].dtype in ['float64']:
        df[var] = (
            df.groupby(df['date'].dt.year)[var]
              .transform(lambda x: x.fillna(x.median()))
        )

In [ ]:
missing_df = (
    df.isna()
      .sum()
      .sort_values(ascending=False)
      .reset_index()
      .set_axis(['variable', 'missing'], axis=1)
)
missing_df['missing_pct'] = missing_df['missing'] / len(df)

missing_df

In [ ]:
for col in continuous_cols:
    nbins = min(50, df[col].nunique())
    fig = px.histogram(
        df,
        x=col,
        nbins=nbins,
        color_discrete_sequence=["cornflowerblue"],
        title=f"Distribution of {col}",
        labels={col: col, "count": "Count"}
    )
    fig.update_layout(
        bargap=0.05,
        template="plotly_white",
        xaxis_title=col,
        yaxis_title="Count"
    )
    fig.show()

Several columns are extremely skewed. Lets see if log1p transformation can reign in variance.

In [ ]:
for c in ["cig_0", "cig_1", "cig_2", "cig_3", 'priorlive', 'priordead', 'priorterm', 'precare']:
  orig_var = df[c].var()
  log_var = np.log1p(df[c]).var()
  print(f"Variance of {c}: {orig_var};  {log_var}")

Some columns are also extremely zero-inflated, so let's break them out into seprate binary and log1p transformed versions to encode both y/n and "magnitude". We can select / prune later on. Let's also skip the transform of priordead and just turn it into a binary - magnitude may not meaningfully add to the model since most have between 0-2.

In [ ]:
df["cig_0_binary"] = (df["cig_0"] > 0).astype(int)
df["cig_0_log1p"] = np.log1p(df["cig_0"])

df["cig_1_binary"] = (df["cig_1"] > 0).astype(int)
df["cig_1_log1p"] = np.log1p(df["cig_1"])

df["cig_2_binary"] = (df["cig_2"] > 0).astype(int)
df["cig_2_log1p"] = np.log1p(df["cig_2"])

df["cig_3_binary"] = (df["cig_3"] > 0).astype(int)
df["cig_3_log1p"] = np.log1p(df["cig_3"])

df["priorlive_binary"] = (df["priorlive"] > 0).astype(int)
df["priorlive_log1p"] = np.log1p(df["priorlive"])

df["priordead_binary"] = (df["priordead"] > 0).astype(int)

df["priorterm_binary"] = (df["priorterm"] > 0).astype(int)
df["priorterm_log1p"] = np.log1p(df["priorterm"])

df["precare_binary"] = (df["precare"] > 0).astype(int)
df["precare_log1p"] = np.log1p(df["precare"])

to_remove = ["cig_0", "cig_1", "cig_2", "cig_3", "priorlive", "priordead", "priorterm", "precare"]
continuous_cols = [col for col in continuous_cols if col not in to_remove]
continuous_cols.extend(["cig_0_log1p", "cig_1_log1p", "cig_2_log1p", "cig_3_log1p", "priorlive_log1p", "priorterm_log1p", "precare_log1p"])
binary_cols.extend(["cig_0_binary", "cig_1_binary", "cig_2_binary", "cig_3_binary", "priorlive_binary", "priordead_binary", "priorterm_binary", "precare_binary"])

df = df.drop(columns=to_remove)

<h2>Correlation with Pearson Coefficient (Linear Relationships)</h2>

- Works well between:
    - Two continuous variables.
    - A binary and a continuous variable (interpretable as difference in means).
    - Between two binaries, Pearson is equivalent to the phi coefficient, which is basically a normalized chi-square.

In [ ]:
cat_cols_all = [
    col
    for col in df.columns
    if any(substring in col for substring in cat_cols)
    and re.search(r'_\d+$', col)
]

In [ ]:
corr_matrix = df[continuous_cols + binary_cols + cat_cols_all + ['death_reported']].corr(numeric_only=True)

# lower triangle
tri_lower_mask = np.tril(np.ones_like(corr_matrix, dtype=bool), k=-1)
corr_long = (
    corr_matrix.where(tri_lower_mask)
            .stack() # to long form
            .reset_index()
            .rename(columns={'level_0':'variable_1', 'level_1':'variable_2', 0:'corr'})
)

corr_long['abs_corr'] = corr_long['corr'].abs()
top_corr = corr_long.sort_values('abs_corr', ascending=False).reset_index(drop=True)

var_to_name_map = json.load(open(BASE_DIR / "var_name_map.json"))

top_corr['variable_1_full'] = top_corr['variable_1'].map(var_to_name_map)
top_corr['variable_2_full'] = top_corr['variable_2'].map(var_to_name_map)

top_corr = top_corr[['variable_1', 'variable_2', 'variable_1_full', 'variable_2_full', 'corr', 'abs_corr']]

top_corr

In [ ]:
top_corr_with_outcome = top_corr[(top_corr['variable_2'] == 'death_reported') | (top_corr['variable_1'] == 'death_reported')]

top_corr_with_outcome

In [ ]:
tri_upper_mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)
corr_masked = corr_matrix.mask(tri_upper_mask)

fig = px.imshow(
    corr_masked,
    text_auto=".2f",
    color_continuous_scale="RdBu_r",
    zmin=-1, zmax=1,
    title="Linear Correlations (Continuous + One-Hot Variables)",
    aspect="auto"
)

fig.update_layout(
    template="plotly_white",
    coloraxis_colorbar=dict(title="Pearson r"),
    width=1500,
    height=900
)

fig.show()

<h2>Clustering Analysis</h2>

Factor Analysis of Mixed Data (FAMD) - Dimensionality reduction technique designed specifically for datasets that contain both categorical and continuous variables. For linear associations.

Sort of a hybrid of:

- PCA for continuous variables.
- MCA (Multiple Correspondence Analysis) for categorical variables.

In [ ]:
# Not using the expanding scaler here, as this is not a supervised task - we're not trying to predict the outcome.
# Fine if the entirety of the data is used at once for mean and std.
X_num = StandardScaler().fit_transform(df[continuous_cols])

pca = PCA(n_components=2).fit_transform(X_num)
mca = TruncatedSVD(n_components=2).fit_transform(df[binary_cols + cat_cols_all])

pca_3d = PCA(n_components=3).fit_transform(X_num)
mca_3d = TruncatedSVD(n_components=3).fit_transform(df[binary_cols + cat_cols_all])

X_famd_like = np.hstack([pca, mca])
X_famd_like_3d = np.hstack([pca_3d, mca_3d])

# Too compute heavy - stalled even on my local.
#agg = AgglomerativeClustering(n_clusters=4, linkage='ward')
# labels_agg = agg.fit_predict(X_famd_like)

kmeans = KMeans(n_clusters=3, random_state=42)
labels_km = kmeans.fit_predict(X_famd_like)

kmeans_3d = KMeans(n_clusters=3, random_state=42)
labels_km_3d = kmeans.fit_predict(X_famd_like_3d)

In [ ]:
cluster_str = labels_km.astype(str)

fig = px.scatter(
    x=X_famd_like[:, 0],
    y=X_famd_like[:, 1],
    color=cluster_str,
    labels={
        'x': 'Component 1',
        'y': 'Component 2',
        'color': 'Cluster'
    },
    title='FAMD-like Embedding: PCA + SVD',
)

fig.update_layout(
    template="plotly_white",
    width=900,
    height=700
)

fig.show()

NOTE: 3D charts are saved via .html files and code blocks are commented out - they use a lot of RAM since they're interactive, and cause freezing up if rendered in-place.

In [ ]:
'''
df_plot = pd.DataFrame(X_famd_like[:, :3], columns=['Comp1', 'Comp2', 'Comp3'])

fig = px.scatter_3d(
    df_plot,
    x='Comp1', y='Comp2', z='Comp3',
    color=cluster_str,
    color_continuous_scale='Viridis',
    opacity=0.8,
    title='3D FAMD-esque Embedding',
    labels={
        'color': 'Cluster'
    },
)

fig.update_traces(marker=dict(size=5))
# fig.write_html("famd_embedding_cluster_labeled.html")
fig.show()
'''

In [ ]:
scatter_y_series = df['death_reported'].replace({0: 'No', 1: 'Yes'})

fig = px.scatter(
    x=X_famd_like[:, 0],
    y=X_famd_like[:, 1],
    color=scatter_y_series,
    labels={
        'x': 'Component 1',
        'y': 'Component 2',
        'color': 'Death Reported'
    },
    title='FAMD-like Embedding: PCA + SVD',
    color_discrete_map={'No': 'cornflowerblue', 'Yes': 'tomato'}
)

fig.update_layout(
    template="plotly_white",
    width=900,
    height=700
)

fig.show()

So there turns out to be something interesting here when we scrap the clustering algorithm and use the 'Death Reported' outcome variable as the colormap. We see a grouping of the 'Yeses', visible in both the 2D and 3D scatterplots. Though we aren't getting much new information here (this is structure we know about), what seems apparent is that there is the opportunity for a sort of rough decision-boundary, which means there may be enough linear signal here to roll with Logistic Regression. If they were scattered throughout and perfectly intertwined, I would say maybe lets try UMAP/HDBSCAN to identify non-linear structure, or default to a non-linear ML model for the classification task. But it looks like a solid candidtae for Logistic Regression right now!

In [ ]:
'''
fig = px.scatter_3d(
    df_plot,
    x='Comp1', y='Comp2', z='Comp3',
    color=scatter_y_series,
    color_continuous_scale='Viridis',
    opacity=0.8,
    title='3D FAMD-esque Embedding',
    labels={
        'color': 'Death Reported'
    },
)

fig.update_traces(marker=dict(size=5))
# fig.write_html("famd_embedding_outcome_labeled.html")
fig.show()
'''

<h1>Feature Engineering & Selection</h1>

I was going to include these in EDA, however I think to keep things clean, it might make more sense to break these out into a separate notebook, which I will work on next.

LinearFeatureSelector was kept here and I dropped in our expanding scaler as well. The Selector was adapted slightly from its original use case to work with this project and the expanded scaler. Will answer any questions about this, since I wrote them myself :) But high-level, the goal is to generate a diverse set of possible feature sets, with some that generated better R^2 values over F1 and Condition Number, some with better F1 than the other two metrics, and some with a better Condition Number than the others.

So this does not output a single ideal, because in this optimization problem (Multi-Objective or 'MOO'), there isn't one.

Why MOO?

This is a causal model, so classification performance (F1) is not everything. It should do well, but that not enough. We also want goodness-of-fit (R^2) and we want to avoid too much multicollinearity (Kappa / Condition Number). So this gives us combinations of features to explore with different performance skews. We can combine these options with what domain expertise and our learned DAG tell us to make better association / quasi-causal statments.

<h2>Build-a-Dag Workshop</h2>

<h2>Interaction Exploration</h2>

<h2>Feature Selection</h2>

In [ ]:
def global_expanding_standard_scaler_by_date(df, date_col, merge_cols=None, min_periods=1):
    """
    Expanding z-score scaling by unique date (allows multiple data points per date).
    Prevents leakage by using only past data.

    merge_cols is a list of columns that are not scaled. They're called this because
    they're merged back to the columns with the date_col.
    """
    if merge_cols is None:
        merge_cols = []

    df = df.sort_values(date_col)

    feature_cols = [col for col in df.columns if col !=
                    date_col and col not in merge_cols]

    means_map = {}
    stds_map = {}

    unique_dates = df[date_col].drop_duplicates().sort_values().to_numpy()
    for i, current_date in enumerate(unique_dates):
        if i < min_periods:
            continue

        past_data = df[df[date_col] <= current_date][feature_cols].to_numpy()
        if past_data.shape[0] == 0:
            continue
        # If there is only one feature column, must be reshapped to return compatible shape.
        if past_data.ndim == 1:
            if len(feature_cols) == 1:
                past_data = past_data.reshape(-1, 1)
            else:
                raise ValueError(
                    f"Expected {len(feature_cols)} features but got 1D array on {current_date}")

        if past_data.shape[1] != len(feature_cols):
            raise ValueError(
                f"Feature count mismatch at {current_date}: expected {len(feature_cols)} and got {past_data.shape[1]}. Check for duplicates.")

        means = np.mean(past_data, axis=0)
        stds = np.std(past_data, axis=0, ddof=0)
        means_map[current_date] = means
        stds_map[current_date] = np.where(stds == 0, 1e-8, stds)

    scaled_array = np.full((len(df), len(feature_cols)), np.nan)

    date_to_index = {d: np.where(df[date_col] == d)[0] for d in unique_dates}

    for i, current_date in enumerate(unique_dates):
        if current_date not in means_map:
            continue
        idx = date_to_index[current_date]
        X = df.iloc[idx][feature_cols].to_numpy()
        scaled = (X - means_map[current_date]) / stds_map[current_date]
        scaled_array[idx, :] = scaled

    scaled_df = pd.DataFrame(scaled_array, columns=feature_cols)
    scaled_df[[date_col] + merge_cols] = df[[date_col] + merge_cols].values
    scaled_df = scaled_df.dropna(subset=feature_cols, how="all", axis=0)

    return scaled_df

class LinearFeatureSelector:
    def __init__(self, n_trials=200, random_state=42):
        self.n_trials = n_trials
        self.random_state = random_state

    def _eval_subset(self, X, y, binary_vars, date_col, cols):
        # if empty subset, return terrible scores so NSGA-II moves away from it
        if len(cols) == 0:
            return -1e6, 1e6, 1e6
        
        if binary_vars is None:
            binary_vars = []

        X_sub = X[cols]

        cv = TimeSeriesSplit(n_splits=5, test_size=4, gap=0)

        f1s = []
        r2s = []
        for train_index, test_index in cv.split(X_sub):
            X_train, X_test = X_sub.iloc[train_index], X_sub.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            X_train_needs_scaling = X_train[[col for col in X_train.columns if col not in binary_vars]]
            X_test_needs_scaling = X_test[[col for col in X_test.columns if col not in binary_vars]]
            
            X_train_scaled = global_expanding_standard_scaler_by_date(X_train_needs_scaling, date_col, min_periods=1)
            X_test_scaled = global_expanding_standard_scaler_by_date(X_test_needs_scaling, date_col, min_periods=1)

            X_train_scaled_with_unscaled = pd.concat([X_train_scaled, X_train[[col for col in X_train.columns if col in binary_vars]]], axis=1)
            X_test_scaled_with_unscaled = pd.concat([X_test_scaled, X_test[[col for col in X_test.columns if col in binary_vars]]], axis=1)

            lr = LogisticRegression(n_jobs=-1, random_state=self.random_state).fit(X_train_scaled_with_unscaled, y_train)
            
            yhat = lr.predict(X_test_scaled_with_unscaled)
            
            in_sample_r2 = r2_score(y_test, yhat)
            oos_f1 = f1_score(y_test, yhat)
            r2s.append(in_sample_r2)
            f1s.append(oos_f1)

        avg_r2 = float(np.mean(r2s))
        avg_f1 = float(np.mean(f1s))

        X_group_needs_scaling = X_sub[[col for col in X_sub.columns if col not in binary_vars]]
        X_group_scaled = global_expanding_standard_scaler_by_date(X_group_needs_scaling, "date", ["date"], min_periods=1)
        X_group_scaled_with_unscaled = pd.concat([X_group_scaled, X_sub[[col for col in X_sub.columns if col in binary_vars]]], axis=1)
        condition_number_kappa = np.linalg.cond(sm.add_constant(X_group_scaled_with_unscaled))

        return avg_r2, avg_f1, condition_number_kappa


    def _run_nsga_feature_search(self, X, y, binary_vars, established_model_variables, date_col):

        df = pd.concat([X, y], axis=1).dropna(axis=1, how="all").dropna()
        X = df[X.columns.intersection(df.columns)]
        y = df[y.name] if getattr(y, "name", None) else df.iloc[:, -1]

        established_model_variables = established_model_variables or []
        for c in established_model_variables:
            if c not in X.columns:
                raise ValueError(f"Control variable '{c}' not in X.")

        candidate_cols = [c for c in X.columns if c not in established_model_variables]

        def __objective(trial: optuna.Trial):
            chosen_cols = []
            for col in candidate_cols:
                use_col = trial.suggest_categorical(f"use::{col}", [0, 1])
                if use_col == 1:
                    chosen_cols.append(col)

            eval_set = established_model_variables + chosen_cols

            r2, f1, kappa = self._eval_subset(X, y, binary_vars, date_col, eval_set)

            trial.set_user_attr("cols", chosen_cols)

            return r2, f1, kappa

        study = optuna.create_study(
            directions=["maximize", "maximize", "minimize"], # R2 higher better, F1 higher better, kappa (condiition number) lower better
            sampler=optuna.samplers.NSGAIISampler(seed=self.random_state),
        )
        study.optimize(__objective, n_trials=self.n_trials)

        return study
    
    def _build_pareto_front_table(self, study):
        rows = []
        # Each trial contains a candidate set of features; we don't scrap any.
        # Result is a Pareto Front, with the top being the non-dominated ones.
        for tr in study.best_trials:
            r2, f1, kappa = tr.values
            cols = tr.user_attrs.get("cols", [])
            rows.append(
                {
                    "trial_id": tr.number,
                    "r2": r2,
                    "f1": f1,
                    "kappa": kappa,
                    "n_features": len(cols),
                    "features": cols,
                }
            )
        df_pf = pd.DataFrame(rows).sort_values(
            ["r2", "f1", "kappa"], ascending=[False, False, True]
        )
        return df_pf.reset_index(drop=True)
    
    def fit(self, X, y, date_col, binary_vars=None, established_model_variables=None):
        study = self._run_nsga_feature_search(X, y, binary_vars, established_model_variables, date_col)
        pareto_table = self._build_pareto_front_table(study)
        return pareto_table